# Example of how to access Trends.Earth API for LandPKS

In [1]:
import json
import requests
import time
from getpass import getpass

import ipyplot
from IPython.display import Image

##  Login

In [2]:
api_url = "https://api.trends.earth"
email, password = input('Email: '), getpass('Password: ')
creds = { "email" : email, "password" : password }
auth_url = api_url + "/auth"

auth_resp = requests.post(auth_url, json=creds)
if auth_resp.status_code == 200:
    print("Logged in as {}.".format(email))
    token = auth_resp.json()['access_token']
else:
    print("Error logging in. Sever returned {}.".format(auth_resp.status_code))

Email:  azvoleff@gmail.com
Password:  ····················


Logged in as azvoleff@gmail.com.


## Submit task and poll for results

In [12]:
start_time = time.time()
params = {'user': 'azvoleff@gmail.com',
          'lang': 'en',
          "year_start": 2001,
          "year_end": 2019,
          "geojson": "{\"coordinates\": [36.43006246, -0.71011347], \"type\": \"Point\"}",
         }
resp_run = requests.post(api_url + '/api/v1/script/landpks-1-0-3/run',
                         json=params,
                         headers={'Authorization' : 'Bearer {}'.format(token)})
print('Submission took {} seconds'.format(time.time() - start_time))

def get_results(id):
    n = 0
    while True:
        resp = requests.get(api_url + '/api/v1/execution/{}'.format(id),
                            json={'user': 'azvoleff@gmail.com'},
                            headers={'Authorization' : 'Bearer {}'.format(token)})
        if resp.json()['data']['status'] == 'FINISHED':
            return resp.json()
        elif resp.json()['data']['status'] == 'FAILED':
            return 'Failed'
        elif n > 400:
            return 'Timeout'
        time.sleep(0.2)
        n += 1
print(resp_run)
        
res = get_results(resp_run.json()['data']['id'])
print('Processing took {} seconds'.format(time.time() - start_time))
print('Return from server:')
# The required data is in the 'results' dictionary. Images will be deleted after 14 days.
#print(json.dumps(res['data']['results'], indent=4))
images = [res['data']['results']['landtrend_plot']['url']['url'],
          res['data']['results']['base_image']['url']['url'],
          res['data']['results']['greenness']['url']['url'],
          res['data']['results']['greenness_trend']['url']['url']]
ipyplot.plot_images(images, img_width=600)

Submission took 0.07634687423706055 seconds
<Response [200]>
Processing took 99.36019396781921 seconds
Return from server:


In [9]:
print(res)
print(json.dumps(res['data']['results'], indent=4))

Failed


TypeError: string indices must be integers